In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 4.5 MB/s 
     |████████████████████████████████| 61 kB 429 kB/s 
     |████████████████████████████████| 895 kB 54.9 MB/s 
     |████████████████████████████████| 3.3 MB 39.4 MB/s 
     |████████████████████████████████| 596 kB 76.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install datasets

     |████████████████████████████████| 306 kB 4.3 MB/s 
     |████████████████████████████████| 243 kB 51.8 MB/s 
     |████████████████████████████████| 1.1 MB 44.3 MB/s 
     |████████████████████████████████| 132 kB 47.5 MB/s 
     |████████████████████████████████| 160 kB 56.6 MB/s 
     |████████████████████████████████| 192 kB 42.7 MB/s 
     |████████████████████████████████| 271 kB 55.5 MB/s 


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

url = 